In [1]:
using DynamicHMCModels

ProjDir = rel_path_d("..", "scripts", "12")

df = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(df) # Should be 10x5

(10, 5)

New col logpop, set log() for population data

In [2]:
df[:society] = 1:10;
df[:logpop] = map((x) -> log(x), df[:population]);
df[:total_tools] = convert(Vector{Int64}, df[:total_tools])
first(df[[:total_tools, :logpop, :society]], 5)

,total_tools,logpop,society
,Int64,Float64,Int64
1,13,7.00307,1
2,22,7.31322,2
3,24,8.18869,3
4,43,8.47449,4
5,33,8.90924,5


Define problem data structure

In [3]:
struct m_12_06d{TY <: AbstractVector, TX <: AbstractMatrix,
  TS <: AbstractVector}
    "Observations (total_tools)."
    y::TY
    "Covariates (logpop)"
    X::TX
    "Society"
    S::TS
    "Number of observations (10)"
    N::Int
    "Number of societies (also 10)"
    N_societies::Int
end;

Make the type callable with the parameters *as a single argument*.

In [4]:
function (problem::m_12_06d)(θ)
    @unpack y, X, S, N, N_societies = problem   # extract the data
    @unpack β, α, s = trans(θ)  # β : a, bp, α : a_society, s
    σ = s[1]^2
    ll = 0.0
    ll += logpdf(Cauchy(0, 1), σ) # sigma
    ll += sum(logpdf.(Normal(0, σ), α)) # α[1:10]
    ll += logpdf.(Normal(0, 10), β[1]) # a
    ll += logpdf.(Normal(0, 1), β[2]) # bp
    ll += sum(
      [loglikelihood(Poisson(exp(α[S[i]] + dot(X[i, :], β))), [y[i]]) for i in 1:N]
    )
end

Instantiate the model with data and inits.

In [5]:
N = size(df, 1)
N_societies = length(unique(df[:society]))
X = hcat(ones(Int64, N), df[:logpop]);
S = df[:society];
y = df[:total_tools];
γ = (β = [1.0, 0.25], α = rand(Normal(0, 1), N_societies), s = [0.2]);
p = m_12_06d(y, X, S, N, N_societies);

Function convert from a single vector of parms to parks NamedTuple

In [6]:
trans = as((β = as(Array, 2), α = as(Array, 10), s = as(Array, 1)));

Define input parameter vector

In [7]:
θ = inverse(trans, γ);
p(θ)

-4498.8772119928535

Maximum_a_posterior

In [8]:
using Optim

x0 = θ;
lower = vcat([0.0, 0.0], -3ones(10), [0.0]);
upper = vcat([2.0, 1.0], 3ones(10), [5.0]);
ll(x) = -p(x);

inner_optimizer = GradientDescent()

res = optimize(ll, lower, upper, x0, Fminbox(inner_optimizer));
res

Results of Optimization Algorithm
 * Algorithm: Fminbox with Gradient Descent
 * Starting Point: [1.0,0.25, ...]
 * Minimizer: [1.0643859799067346,0.2663911664386152, ...]
 * Minimum: -1.300258e+02
 * Iterations: 1000
 * Convergence: false
   * |x - x'| ≤ 0.0e+00: false 
     |x - x'| = 1.65e-10 
   * |f(x) - f(x')| ≤ 0.0e+00 |f(x)|: false
     |f(x) - f(x')| = -2.41e-07 |f(x)|
   * |g(x)| ≤ 1.0e-08: false 
     |g(x)| = 1.98e+05 
   * Stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 428818295
 * Gradient Calls: 428818295

Minimum gives MAP estimate:

In [9]:
Optim.minimizer(res)

13-element Array{Float64,1}:
  1.0643859799067346    
  0.2663911664386152    
  8.523780367480687e-13 
 -1.7030530296947533e-13
  3.3438621832080745e-13
 -2.0220918887814007e-12
 -1.1169296121859496e-13
  1.9481694561307964e-12
 -8.093796086823485e-13 
  1.3649549719389436e-12
 -1.9507585057782423e-12
  1.4383018110050093e-11
  0.00010129358984945946

Write a function to return properly dimensioned transformation.

In [10]:
problem_transformation(p::m_12_06d) =
  as( Vector, length(θ) )
# Wrap the problem with a transformation, then use ForwardDiff for the gradient.
P = TransformedLogDensity(problem_transformation(p), p)
∇P = LogDensityRejectErrors(ADgradient(:ForwardDiff, P));
#∇P = ADgradient(:ForwardDiff, P);

Tune and sample.

In [11]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000);

MCMC, adapting ϵ (75 steps)
0.0036 s/step ...done
MCMC, adapting ϵ (25 steps)
0.0033 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0033 s/step ...done
MCMC, adapting ϵ (100 steps)
0.0018 s/step ...done
MCMC, adapting ϵ (200 steps)
0.0011 s/step ...done
MCMC, adapting ϵ (400 steps)
0.00071 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0008 s/step ...done
MCMC (1000 steps)
0.00058 s/step ...done


We use the transformation to obtain the posterior from the chain.

In [12]:
posterior = TransformVariables.transform.(Ref(problem_transformation(p)), get_position.(chain));
posterior[1:5]

5-element Array{Array{Float64,1},1}:
 [1.29634, 0.239938, -0.0446824, 0.151847, -0.00437711, 0.0717697, 0.0372942, 0.00610078, 0.0161328, 0.00859503, 0.162651, -0.0828914, -0.295605]
 [1.46497, 0.22847, -0.0739989, -0.0704028, 0.0360656, 0.0720681, 0.0534791, -0.0945842, -0.0747204, -0.0945628, 0.0749444, 0.0475107, -0.275903]
 [1.25989, 0.247037, -0.135905, 0.0848726, -0.00821491, 0.155153, 0.176456, -0.081402, 0.0306032, -0.13193, 0.149093, -0.0962582, -0.338526]     
 [0.716891, 0.299607, -0.247872, 0.124827, -0.101085, 0.330114, -0.118024, -0.500061, 0.139611, -0.157173, 0.285865, -0.21794, -0.632238]        
 [1.22251, 0.262667, -0.0237674, 0.0321429, -0.369004, 0.192322, -0.109753, -0.565714, 0.350075, -0.163937, 0.253706, -0.140848, -0.505079]      

Extract the parameter posterior means.

In [13]:
posterior_β = mean(trans(posterior[i]).β for i in 1:length(posterior))
posterior_α = mean(trans(posterior[i]).α for i in 1:length(posterior))
posterior_σ = mean(trans(posterior[i]).s for i in 1:length(posterior))[1]^2

0.2778870225435048

Effective sample sizes (of untransformed draws)

In [14]:
ess = mapslices(effective_sample_size, get_position_matrix(chain); dims = 1)
ess

1×13 Array{Float64,2}:
 853.235  810.277  714.791  952.704  …  609.847  442.33  686.761  352.285

NUTS-specific statistics

In [15]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.93, min/25%/median/75%/max: 0.25 0.91 0.97 0.99 1.0
  termination: AdjacentTurn => 4% DoubledTurn => 96%
  depth: 2 => 0% 3 => 5% 4 => 94% 5 => 0%


CmdStan result

In [16]:
m_12_6_result = "
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
                            Mean                SD               Naive SE             MCSE            ESS
            a          1.076167468  0.7704872560 0.01218247319 0.0210530022 1000.000000
           bp         0.263056273  0.0823415805 0.00130193470 0.0022645077 1000.000000
  a_society.1   -0.191723568  0.2421382537 0.00382854195 0.0060563054 1000.000000
  a_society.2    0.054569029  0.2278506876 0.00360263570 0.0051693148 1000.000000
  a_society.3   -0.035935050  0.1926364647 0.00304584994 0.0039948433 1000.000000
  a_society.4    0.334355037  0.1929971201 0.00305155241 0.0063871707  913.029080
  a_society.5    0.049747513  0.1801287716 0.00284808595 0.0043631095 1000.000000
  a_society.6   -0.311903245  0.2096126337 0.00331426674 0.0053000536 1000.000000
  a_society.7    0.148637507  0.1744680594 0.00275858223 0.0047660246 1000.000000
  a_society.8   -0.164567976  0.1821341074 0.00287979309 0.0034297298 1000.000000
  a_society.9    0.277066965  0.1758237250 0.00278001719 0.0055844175  991.286501
 a_society.10   -0.094149204  0.2846206232 0.00450024719 0.0080735022 1000.000000
sigma_society    0.310352849  0.1374834682 0.00217380450 0.0057325226  575.187461
";

Show means

In [17]:
[posterior_β, posterior_α, posterior_σ]

3-element Array{Any,1}:
  [1.11507, 0.259458]                                                                                          
  [-0.185776, 0.0459235, -0.0382586, 0.319526, 0.0477397, -0.296581, 0.146769, -0.159159, 0.267409, -0.0886024]
 0.2778870225435048                                                                                            

End of m12.6d1.jl

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*